In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

## Data Prep ##

In [2]:
sofr_raw = pd.read_excel("hist_data.xlsm", sheet_name="SofrCurve")
aapl_raw = pd.read_excel("hist_data.xlsm", sheet_name="AAPL").set_index("Date")
msft_raw = pd.read_excel("hist_data.xlsm", sheet_name="MSFT").set_index("Date")
f_raw = pd.read_excel("hist_data.xlsm", sheet_name="F").set_index("Date")
bac_raw = pd.read_excel("hist_data.xlsm", sheet_name="BAC").set_index("Date")

In [3]:
# clean up sofr raw
sofr_raw = sofr_raw.T
annualized_time = sofr_raw.loc["T"]
sofr_raw.drop(["T"], inplace=True)
sofr_raw.columns = sofr_raw.loc["Tenor"]
sofr_raw.drop(["Tenor"], inplace=True)
sofr_raw.columns.name = "Date"

In [4]:
sofr_raw

Date,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-10-31,0.039191,0.038721,0.03867,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.03235,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.04697,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.03601,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.0391,0.040852,0.044884,0.04658,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.03935,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.0475,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.03738,0.037152,0.03687,0.036534,0.034424,0.032558,0.030723,0.028933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.053105,0.053084,0.053202,0.053424,0.05369,0.053293,0.052503,0.048399,0.045999,0.04485,...,0.044093,0.044056,0.043972,0.043841,0.043663,0.043437,0.041874,0.040238,0.038438,0.036493
2023-10-25,0.052981,0.053057,0.053259,0.05349,0.053749,0.053393,0.052653,0.048791,0.046595,0.045594,...,0.045289,0.045268,0.045198,0.045078,0.044906,0.044683,0.043076,0.041358,0.039564,0.037641
2023-10-26,0.053047,0.053089,0.053218,0.053414,0.053567,0.053089,0.052243,0.048044,0.045645,0.044538,...,0.044269,0.044244,0.044171,0.044049,0.043882,0.043668,0.042182,0.040589,0.03876,0.036764
2023-10-27,0.052989,0.05304,0.053185,0.053368,0.053486,0.052991,0.052115,0.047758,0.045284,0.0442,...,0.04446,0.044461,0.044411,0.044309,0.044157,0.043955,0.042508,0.040985,0.039167,0.037148


In [5]:
aapl_raw.rename(columns={"Adj Close": "adj_close_aapl"}, inplace=True)
msft_raw.rename(columns={"Adj Close": "adj_close_msft"}, inplace=True)
f_raw.rename(columns={"Adj Close": "adj_close_f"}, inplace=True)
bac_raw.rename(columns={"Adj Close": "adj_close_bac"}, inplace=True)

In [6]:
aapl_raw

,adj_close_aapl
Date,
2022-10-31,152.041122
2022-11-01,149.373917
2022-11-02,143.801514
2022-11-03,137.703613
2022-11-04,137.435455
...,...
2023-10-24,172.991058
2023-10-25,170.657135
2023-10-26,166.458023


In [7]:
sofr_diff = sofr_raw.pct_change()
aapl_diff = aapl_raw.pct_change()
msft_diff = msft_raw.pct_change()
f_diff = f_raw.pct_change()
bac_diff = bac_raw.pct_change()

In [8]:
sofr_diff

Date,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-01,0.010547,0.007807,0.005586,0.004642,0.006097,0.009650,0.011215,0.009846,0.008140,0.007351,...,-0.009401,-0.010132,-0.010714,-0.011198,-0.011620,-0.012014,-0.013077,-0.011484,-0.010291,-0.008010
2022-11-02,0.008690,0.006724,0.005495,0.003134,0.000791,0.002834,0.004977,0.010528,0.009553,0.006256,...,0.001441,0.000524,-0.000355,-0.001092,-0.001598,-0.001803,-0.000542,-0.001341,0.001790,0.004557
2022-11-03,0.011023,0.007620,0.006400,0.007383,0.008847,0.011307,0.014624,0.024133,0.025373,0.023414,...,0.009925,0.010021,0.010619,0.011287,0.011670,0.011476,0.005465,0.006191,0.003812,0.001641
2022-11-04,0.138063,0.069681,-0.014103,-0.013198,-0.001532,-0.007527,-0.008229,-0.010659,-0.010219,-0.007710,...,0.012528,0.013177,0.013401,0.013495,0.013697,0.014205,0.018141,0.013190,0.010363,0.009691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.000273,0.000130,0.000208,0.000365,0.000964,0.001556,0.002179,0.007733,0.009140,0.007695,...,-0.003648,-0.003932,-0.004137,-0.004304,-0.004467,-0.004654,-0.006054,-0.007345,-0.007298,-0.006589
2023-10-25,-0.002326,-0.000512,0.001074,0.001242,0.001101,0.001871,0.002860,0.008113,0.012954,0.016572,...,0.027122,0.027519,0.027886,0.028209,0.028481,0.028693,0.028704,0.027814,0.029305,0.031443
2023-10-26,0.001249,0.000597,-0.000761,-0.001429,-0.003381,-0.005693,-0.007787,-0.015316,-0.020383,-0.023143,...,-0.022520,-0.022618,-0.022730,-0.022809,-0.022818,-0.022726,-0.020773,-0.018590,-0.020321,-0.023303
2023-10-27,-0.001109,-0.000924,-0.000628,-0.000855,-0.001523,-0.001858,-0.002466,-0.005961,-0.007903,-0.007602,...,0.004318,0.004906,0.005432,0.005888,0.006270,0.006574,0.007743,0.009759,0.010495,0.010447


In [9]:
msft_diff

,adj_close_msft
Date,
2022-10-31,NaN
2022-11-01,-0.017059
2022-11-02,-0.035368
2022-11-03,-0.026579
2022-11-04,0.033326
...,...
2023-10-24,0.003674
2023-10-25,0.030678
2023-10-26,-0.037514


In [10]:
# combine df?
stock_return = pd.concat([aapl_diff, msft_diff, f_diff, bac_diff], axis=1)
stock_return.dropna(inplace=True)
stock_return

,adj_close_aapl,adj_close_msft,adj_close_f,adj_close_bac
Date,,,,
2022-11-01,-0.017543,-0.017059,0.002244,0.004439
2022-11-02,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04,-0.001947,0.033326,0.018854,0.025077
2022-11-07,0.003902,0.029270,0.014064,0.005980
...,...,...,...,...
2023-10-24,0.002543,0.003674,-0.007840,-0.003911
2023-10-25,-0.013492,0.030678,0.013169,0.003141
2023-10-26,-0.024606,-0.037514,-0.016464,0.022309


## Moddeling the Joint Distribution of the Daily Risk Factor Changes ##
* For each risk factor, specify how you want to model their changes. As I suggested, one
way is to model the absolute daily changes in the zero rate risk factors and model the
daily relative returns of the stock prices.
* Once the above is specified, follow the approach of each VaR model (parametric/Monte
Carlo/historical) to model the joint distribution of the daily risk factor changes.

For each risk factor, you would need to use the 1-year historical data provided to compute their
daily changes over the 1-year historical period. Using the historical daily changes of the risk
factors, one can then determine the joint distribution of 1-day risk factor changes under each of
the 3 VaR models

* Parametric VaR. The 1-day risk factor changes are assumed to be multivariate-normally
distributed. One can simply estimate the means and covariance matrix of the multivariate
normal distribution using the sample means and convariance matrix calculated from the
historical daily changes.
* Monte Carlo VaR. The 1-day risk factor changes are also assumed to be
multivariate-normally distributed. Hence, one simply follow the same appproach as the
parametric VaR to estimate the means and covariance matrix of the multivariate normal
distribution.
* Historical VaR. The joint distribution of the 1-day risk factor changes is simply given by
the empirical distribution of the 1-year historical sample of daily risk factor changes.
Dr. Tony Wong 4 / 21


## Stock VaR - Parametric ##

This is kinda following the lecture notes. Use the percentage diff

In [11]:
# params
stock_return_mean = stock_return.mean()
stock_return_cov = stock_return.cov()
stock_w = 1e6 * np.ones(4)

Prof Tony made a typo in the lecture notes, recall from QF6000 asset pricing
* portfolio mean return = w`R
* portfolio variance = w`Vw

Where w = weights of the portfolio, V is return covariance matrix, R is the return matrix

In [12]:
# my instinct was to use np.dot, but this is easier to read & understand
stock_port_mean = np.matmul(stock_w, stock_return_mean)
stock_port_var = np.matmul(np.matmul(stock_w, stock_return_cov), stock_w)

In [13]:
z_value = norm.ppf(0.05)

Z-value formula adapted to our convention for VaR calculation is
$$z = \frac{J-\mu}{\sigma}$$
VaR is deined as
$$ VaR_{\alpha,h} = |J|$$
$$ P(L \leq J) = \alpha $$
The project wants one-tail 95% confidence, so that means $$\alpha = 0.05$$
That means
$$J = |\mu + \sigma \cdot z_{\alpha=0.05}|

In [14]:
var_stock = abs(stock_port_mean + z_value * np.sqrt(stock_port_var))
var_stock

87730.49630401182

## Swap VaR - Parametric ##
